In [91]:
from IPython.display import clear_output
import html2text
import re

from collections import Counter
from itertools import chain


NUMBER_OF_ARTICLES = 882
DIVISION = "--------------"
LEN_DIVISION = len(DIVISION)

sizes = []

for id in range(NUMBER_OF_ARTICLES):
    with open("./raw_articles/article_"+str(id)+".txt", "r") as fin:
        # print("---------------->",id,"<----------------\n")
        text = fin.read()
        clean = text[text.find(DIVISION)+LEN_DIVISION:]
        # Remover pictures
        clean = re.sub(r'<img[^>]*>', '', clean)
        # Removes javascript from the end
        clean = re.sub(r'<div id="atatags-[^>]*>.*', '', clean, flags=re.DOTALL)
        clean = re.sub(r'<span id="wordads-inline-marker"[^>]*>.*?</span>', '', clean, flags=re.DOTALL)
        clean = re.sub(r'<div id="jp-post-flair[^>]*>.*?</div>', '', clean, flags=re.DOTALL)
        clean = re.sub(r"<div class='sharedaddy[^>]*>.*?</div>", '', clean, flags=re.DOTALL)
        clean = re.sub(r"<div id='jp-relatedposts'[^>]*>.*?</div>", '', clean, flags=re.DOTALL)
        clean = html2text.html2text(clean)
        clean = re.sub(r'Clicar na foto para ampliar e obter mais informações:\s*\[\s*\n\s*\]\(http://www\.23hq\.com.*?\)', '', clean, flags=re.DOTALL)
        clean = re.sub(r'\[\s*\n\s*\]\(http://www\.23hq\.com.*?\)', '', clean, flags=re.DOTALL)
        size_of_text = len(clean.split()) 
        sizes.append(size_of_text)
        if size_of_text >= 3:
            with open("./treated/article_"+str(id)+".txt", "w") as fout:
                fout.write(clean)
    # if input():
    #     break
    # clear_output()

In [12]:
from openai import OpenAI
client = OpenAI(api_key="")

In [116]:
text = """Rio Novo 19 de Agosto

## **[Deve ser 1923]**

Querido irmãozinho! Saudações"""

test_cases = [
    "Rio Novo 21 de agosto de 1928",
    "Rio Novo 19 de Agosto",
    "Some Random Text",
    "Rio Novo 32 de Janeiro de 1928",  # Invalid date, but matches the format
    "Another Place 10 de Junho",
]

patterns = [
    re.compile(r'^[A-Za-z\s]+ \d{1,2} de [A-Za-z]+ de \d{4}(?:\s|$)', re.MULTILINE),
    re.compile(r'^[A-Za-z\s]+ \d{1,2} de [A-Za-z]+(?:\s|$)', re.MULTILINE)
]
    

def is_possible_letter(text):
    paragraphs = text.split('\n')  # Assuming paragraphs are separated by double newline
    if paragraphs:
        first_paragraph = paragraphs[0]
        for pattern in patterns:
            if re.search(pattern, first_paragraph):
                return True


for case in test_cases:
    result = is_possible_letter(case)
    print(f"'{case}': {'Possibly a letter' if result else 'Not a letter'}")

'Rio Novo 21 de agosto de 1928': Possibly a letter
'Rio Novo 19 de Agosto': Possibly a letter
'Some Random Text': Not a letter
'Rio Novo 32 de Janeiro de 1928': Possibly a letter
'Another Place 10 de Junho': Possibly a letter


In [130]:
import os

letters = 0

patterns = [
    re.compile(r'^[A-Za-z\s]+ \d{1,2} de [A-Za-z]+ de \d{4}$'),  # [Place] [Day] de [Month] de [Year]
    re.compile(r'^[A-Za-z\s]+ \d{1,2} de [A-Za-z]+$'),            # [Place] [Day] de [Month]
    re.compile(r'^[A-Za-z\s]+ \d{1,2} de [A-Za-z]+ de \d{4}$'),
    re.compile(r'^\d{1,2} de [A-Za-z]+ de \d{4}$')   
]

for filename in os.listdir('./treated'):
    is_letter = False
    with open("./treated/"+filename, "r") as fin:
        text = fin.read()
        paragraphs = text.split('\n')  # Assuming paragraphs are separated by double newline
        if paragraphs:
            for pattern in patterns:
                if re.search(pattern, paragraphs[0].strip()):
                    is_letter = True
        # print(text)
        if is_letter:
            with open("./letter/"+filename, "w") as fout:
                fout.write(text)
        else:
            with open("./not_letter/"+filename, "w") as fout:
                fout.write(text)
    

In [144]:
responses = {}

for filename in os.listdir('./not_letter'):
    with open("./not_letter/"+filename, "r") as fin:
        text = fin.read()
        print(text)
    res = input("Accept?")
    if res == "123":
        break
    elif res != "0":
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": "Leia o trecho a seguir sobre um post sobre uma Colônia Leta no Brasil e resuma os fatos principais sobre a colônia obtidos nesse post. Mencione datas, se possível:\n" + text,
                }
            ],
            model="gpt-3.5-turbo",
        )
        response = chat_completion.choices[0].message.content
        print(response)
        responses[filename] = response
        input("Confirm GPT response:")
    else:
        with open("./to_summarize/human_reject/"+filename, "w") as fout:
            fout.write(text)
    clear_output()
    

Rio Novo, 24-10-1918

Querido irmãozinho,

Rio de Janeiro

Primeiramente aceite os meus cumprimentos! -- Já faz tempo que não mais tenho
escrito, mas hoje vou escrever um pouco. Eu estou passando bem. Hoje o tempo
está ótimo, mas passou um grande período chuvoso. Não conseguimos queimar as
coivaras e já há bastante verde brotando nas derrubadas. Se tudo der certo,
vamos queimar a grande coivara da mata virgem na semana que vem. Fique lá
atento para o espoucar das taquaras e a grande fumaça, quem sabe com um pouco
de esforço você consiga avistar…

Na semana passada começamos a capinar as roças na Bukovina; as de perto de
casa já tínhamos terminado. Ontem fomos plantar batata inglesa na Bukovina. No
ano passado deu batata muito boa lá, e desta vez plantamos mais. No ano
passado plantamos 1 quarta e colhemos 14 quartas. Algumas semanas atrás levei
o meu [porco] "Nerpukli" para a cidade e rendeu 4 arrobas. Agora estou _muito
rico_ e em dinheiro rendeu 42$000. Se você viesse para casa eu po

Accept? 123


In [146]:
import pickle

with open('gpt_summary.pickle', 'wb') as handle:
    pickle.dump(responses, handle)


In [149]:
with open('gpt_summary.pickle', 'rb') as handle:
    b = pickle.load(handle)

b


{'article_89.txt': 'Resumo:\n- O autor do post está servindo o Exército em Florianópolis sob regime de guerra.\n- Ele viajou de sua cidade natal para Laguna e depois para Florianópolis.\n- A viagem de barco foi tranquila, mas houve pagamento de tributos ao deus do mar.\n- Na caserna em Florianópolis, há cerca de 300 pessoas, sendo que algumas estão chegando e outras estão partindo após cumprir seu tempo.\n- A comida na caserna é satisfatória, com café e pão de manhã, almoço e janta, e um lanche à tarde.\n- O autor espera servir o exército em Curitiba ou Ponta Grossa, pois apenas uma parte dos soldados ficará em Florianópolis.\n- Ele menciona sobre procurar uma igreja presbiteriana e possivelmente se tornar amigo do General Rosinha, que é batista.',
 'article_805.txt': 'A Colônia Letã no Brasil enfrentou diversos problemas causados por queimadas, que destruíram várias casas, matas, capoeiras, pastos, cercas, entre outras áreas. As queimadas afetaram diversas propriedades e regiões, como